<a href="https://colab.research.google.com/github/zjzsu2000/CMPE258/blob/master/Ungraded_assignment_5/4)%20Tesorflow_Redo_4__text_classification_with_hub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

####ref:https://www.tensorflow.org/tutorials/keras/text_classification_with_hub

# 4) Tesorflow_Redo_4_ text_classification_with_hub.ipynb: Movie reviews

This notebook classifies movie reviews as *positive* or *negative* using the text of the review. This is an example of *binary*—classification.


We'll use the [IMDB dataset](https://www.tensorflow.org/api_docs/python/tf/keras/datasets/imdb) that contains the text of 50,000 movie reviews from the [Internet Movie Database](https://www.imdb.com/). These are split into 25,000 reviews for training and 25,000 reviews for testing. 

This notebook uses [tf.keras](https://www.tensorflow.org/guide/keras), a high-level API to build and train models in TensorFlow, and [TensorFlow Hub](https://www.tensorflow.org/hub), a library and platform for transfer learning. 

In [0]:
import numpy as np
import tensorflow as tf

!pip install tensorflow-hub
!pip install tfds-nightly
import tensorflow_hub as hub
import tensorflow_datasets as tfds


In [0]:
print("TensorFlow Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")

## Download IMDB dataset


Split the training set into 60% and 40%, so we'll end up with 15,000 examples for training, 10,000 examples for validation and 25,000 examples for testing.

In [0]:
train_data, valid_data, test_data = tfds.load(name="imdb_reviews", 
                               split=('train[:60%]', 'train[60%:]', 'test'),
                               as_supervised=True)

## Explore the data 



In [0]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

In [0]:
train_labels_batch

## Build the model



Let's first create a Keras layer that uses a TensorFlow Hub model to embed the sentences, and try it out on a couple of input examples. Note that no matter the length of the input text, the output shape of the embeddings is: `(num_examples, embedding_dimension)`.

In [0]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding,input_shape=[],dtype=tf.string, 
                           trainable=True)
hub_layer(train_examples_batch[:3])

In [0]:
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

1. The first layer is a TensorFlow Hub layer. This layer uses a pre-trained Saved Model to map a sentence into its embedding vector. The pre-trained text embedding model that we are using ([google/tf2-preview/gnews-swivel-20dim/1](https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1)) splits the sentence into tokens, embeds each token and then combines the embedding. The resulting dimensions are: `(num_examples, embedding_dimension)`.
2. This fixed-length output vector is piped through a fully-connected (`Dense`) layer with 16 hidden units.
3. The last layer is densely connected with a single output node.

In [0]:
model = Sequential([hub_layer,Dense(16, activation='relu'),Dense(1)])
model.summary()

### Loss function and optimizer

In [0]:
from tensorflow.keras import losses

In [0]:
model.compile(optimizer='adam',loss=losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

## Train the model

Train the model for 20 epochs in mini-batches= 512. 

In [0]:
history = model.fit(train_data.shuffle(10000).batch(512),epochs=20,
                    validation_data=valid_data.batch(512),verbose=1)

## Evaluate the model


In [0]:
results = model.evaluate(test_data.batch(512), verbose=2)

In [21]:
for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

loss: 0.337
accuracy: 0.844
